# UrbanSound8K

## Cnn

In [1]:
import librosa
import numpy as np
import pandas as pd
import random
import keras
from keras.layers import Activation,Dense,Dropout,Conv2D,Flatten,MaxPooling2D
from keras.models import Sequential

# Intro to librosa

# # 1. Get the file path to the included audio example
# filename = librosa.util.example_audio_file()

# # 2. Load the audio as a waveform `y`
# #    Store the sampling rate as `sr`
# y, sr = librosa.load(filename)
# print(y,sr)

# # 3. Run the default beat tracker
# tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
# print(tempo,beat_frames)

# print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

# # 4. Convert the frame indices of beat events into timestamps
# beat_times = librosa.frames_to_time(beat_frames, sr=sr)
# print(beat_times)

Using TensorFlow backend.


In [2]:
import os

In [3]:
#loading data from files and storing as melspectrogram
def load_data(data,names):
    D = []
    c = 0
    for f in names.itertuples():
        c+=1
        file_name = os.path.join(data,f.path)
        y, sr = librosa.load(file_name ,duration=2.97)
        ps = librosa.feature.melspectrogram(y=y, sr=sr)
        if ps.shape == (128,128):
               D.append((ps,f.classID))
        if(c%1000 ==0):print(f.Index)
    return(D)

In [4]:
data = pd.read_csv(r'C:\Users\hello\Desktop\UrbanSound8K\UrbanSound8K\metadata\UrbanSound8K.csv')
data.head(11)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


In [5]:
data.shape

(8732, 8)

In [6]:
# Get data over 3 seconds long
valid_data = data[['slice_file_name', 'fold' ,'classID', 'class']][ data['end']-data['start'] >= 3 ]
valid_data.shape

(7468, 4)

In [7]:
valid_data.head(15)

,slice_file_name,fold,classID,class
1,100263-2-0-117.wav,5,2,children_playing
2,100263-2-0-121.wav,5,2,children_playing
3,100263-2-0-126.wav,5,2,children_playing
4,100263-2-0-137.wav,5,2,children_playing
5,100263-2-0-143.wav,5,2,children_playing
6,100263-2-0-161.wav,5,2,children_playing
7,100263-2-0-3.wav,5,2,children_playing
8,100263-2-0-36.wav,5,2,children_playing
14,100652-3-0-0.wav,2,3,dog_bark
15,100652-3-0-1.wav,2,3,dog_bark


In [8]:
# adding folder and filename to path column
valid_data['path'] = 'fold' + valid_data['fold'].astype('str') + str("\\") + valid_data['slice_file_name'].astype('str')

In [9]:
t = load_data(r'C:\Users\hello\Desktop\UrbanSound8K\UrbanSound8K\audio',valid_data)
len(t)

1070
2277
3528
4683
5857
7084
8179


7467

In [10]:
print(t[0],t[10])

(array([[1.9723278e-02, 4.1040558e-02, 1.8539123e-02, ..., 8.5462835e-03,
        5.3339968e-03, 3.8408160e-02],
       [2.2824360e-02, 3.6766436e-02, 2.0244755e-02, ..., 2.0034175e-02,
        1.5074839e-02, 1.0440601e-02],
       [1.8157125e-02, 3.0914154e-02, 2.6804928e-02, ..., 1.6050415e-02,
        1.7375546e-02, 1.1339896e-02],
       ...,
       [2.8042020e-06, 2.5728361e-06, 3.4476034e-06, ..., 6.0977309e-06,
        1.1406619e-05, 1.2841096e-05],
       [1.4985648e-06, 1.2168863e-06, 2.0153952e-06, ..., 7.5262238e-07,
        1.8069223e-06, 3.9235983e-06],
       [9.5098898e-07, 2.9593167e-07, 7.4605310e-08, ..., 4.4321943e-08,
        4.9451511e-08, 2.0704729e-07]], dtype=float32), 2) (array([[8.4149241e-03, 1.0018934e-02, 5.2789464e-03, ..., 4.8151087e-02,
        3.1068638e-02, 7.1702623e-03],
       [1.1224099e-02, 4.8862332e-03, 2.3080767e-03, ..., 1.1473654e-02,
        9.2293778e-03, 1.0404558e-02],
       [4.2239870e-03, 1.9912729e-03, 1.2221524e-03, ..., 4.1628005e-0

In [11]:
#shuffeling and splitting of dataset
dataset = t
random.shuffle(dataset)
t[:10]
train = dataset[:7000]
test = dataset[7000:]

X_train, y_train = zip(*train)
X_test, y_test = zip(*test)

# Reshape for CNN input
X_train = np.array([x.reshape( (128, 128, 1) ) for x in X_train])
X_test = np.array([x.reshape( (128, 128, 1) ) for x in X_test])

# One-Hot encoding for classes
y_train = np.array(keras.utils.to_categorical(y_train, 10))
y_test = np.array(keras.utils.to_categorical(y_test, 10))

In [12]:
#training cnn model using keras
model = Sequential()
input_shape = (128,128,1)

model.add(Conv2D(24,(5,5),strides = (1,1),input_shape = input_shape))
model.add(MaxPooling2D((4,2),strides = (4,2)))
model.add(Activation('relu'))

model.add(Conv2D(48,(5,5),padding = "valid"))
model.add(MaxPooling2D((4,2),strides = (4,2)))
model.add(Activation('relu'))

model.add(Conv2D(48,(5,5),padding = "valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate = 0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate = 0.5))

model.add(Dense(10))
model.add(Activation('softmax'))


Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.compile(
    optimizer="Adam",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

model.fit(
    x=X_train, 
    y=y_train,
    epochs=12,
    batch_size=128,
    validation_data= (X_test, y_test))

score = model.evaluate(
    x=X_test,
    y=y_test)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 7000 samples, validate on 467 samples
Epoch 1/12
7000/7000 [==============================] - 75s 11ms/step - loss: 2.6403 - accuracy: 0.2159 - val_loss: 2.1013 - val_accuracy: 0.3041
Epoch 2/12
7000/7000 [==============================] - 70s 10ms/step - loss: 2.0791 - accuracy: 0.2843 - val_loss: 1.8803 - val_accuracy: 0.3533
Epoch 3/12
7000/7000 [==============================] - 70s 10ms/step - loss: 1.8424 - accuracy: 0.3541 - val_loss: 1.6610 - val_accuracy: 0.4133
Epoch 4/12
7000/7000 [==============================] - 68s 10ms/step - loss: 1.7013 - accuracy: 0.4020 - val_loss: 1.5261 - val_accuracy: 0.4775
Epoch 5/12
7000/7000 [==============================] - 71s 10ms/step - loss: 1.5837 - accuracy: 0.4509 - val_loss: 1.4104 - val_accuracy: 0.4711
Epoch 6/12
7000/7000 [==============================] - 71s 10ms/step - loss: 1.4592 - accuracy: 0.4836 - val_loss: 1.3581 - val_accuracy: 0.5182
Epoch 7/12
7000/7000 [=======